# Data Wrangling (WeRateDogs Data)

![Image via Boston Magazine](https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd378f_dog-rates-social/dog-rates-social.jpg)

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#gathering">Gathering</a></li>
<li><a href="#assessing">Assessing</a></li>
<li><a href="#cleaning">Cleaning</a></li>
<li><a href="#iterating">Iterating</a></li>
<li><a href="#storing">Storing</a></li>
<li><a href="#acting">Acting</a></li>
<li><a href="#conclusion">Conclusion</a></li>
<li><a href="#references">References</a></li>
</ul>

<a id='intro'></a>
## Introduction
<div style="text-align:justify">
<br>
    <a href="https://twitter.com/dog_rates?lang=en">WeRateDogs</a> is a Twitter account that rates people's dogs with a humorous comment about the dog. Their Twitter account data as many other real world cases does not comes clean, so as part of the Udacity's Data Analyst Nanodegree program the data wrangling process must be applied in order to obtain data that will be useful to create interesting and trustworthy analyses and visualizations of these rates.
</div>

### Objectives

- Gathering: Gather data from three different sources in order to create a master dataset **(WeRateDogs Twitter archive, Image predictions file and Twitter API)**.

- Assessing: Assess both visually and programmatically for quality and tidiness issues. Detect and document at least **8 quality issues and 2 tidiness issues**.

- Cleaning: Clean each of the issues you documented while assessing. **The result should be a high quality and tidy master pandas DataFrame (or DataFrames, if appropriate)**.

- Storing: Store the clean DataFrame(s) in a CSV file with the main one. **Additionally, store the cleaned data in an SQLite database**.

- Acting: Analyze and visualize the wrangled data, **producing at least 3 insights and 1 visualization**.

In [9]:
# prerequisite package imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb


# library for Twitter API
import tweepy
from tweepy import OAuthHandler

# library to handle json
import json
import collections

# library for sql database
import sqlite3

# package to calculate time
from timeit import default_timer as timer

%matplotlib notebook

<a id='gathering'></a>
## Gathering the Data

The first source that will be gather is the WeRateDogs Twitter archive, this file has been downloaded manually from the following url: [Data source 1](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv).

After the download, I transferred the file to the * src * folder located in the main directory of this notebook. Once there we read the file using the `read_csv` function from Pandas.

In [10]:
folder_name = 'src'
twitter_archive = pd.read_csv('src/twitter-archive-enhanced-2.csv')
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


Now the second source that will be gather is the Image Predictions file, which is hosted on [Udacity's servers](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv) and that we have downloaded manually before hand.

In [11]:
#we just have to read the file, using pandas
image_predictions = pd.read_csv('src/image-predictions-3.tsv', sep="\t")
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


Finally the third source will be gather using Twitter's API, the library `tweepy` allow us to to interact with the API using Python.

In [12]:
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = twitter_archive.tweet_id.values
len(tweet_ids)

2356

In [ ]:
# The tokens for authetication must be HIDDEN for security reasons and
# comply with Twitter's Terms and Conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_token_secret = 'HIDDEN'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Then we create an instance of the API
api = tweepy.API(auth, wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)

In [ ]:
# Next we extract the ids from twitter-archive-enhanced dataset
query_ids = twitter_archive['tweet_id'].values
tweets = len(query_ids)

In [ ]:
# Once having all the ids, we query Twitter's API for information from each Tweet
# Twitter will return JSON data, so we must dump it into a file

# These variables will help to keep on check on how is going the process
# and retrieve failed tweets
curr = 0
start = timer()
failed_tweets = {}

with open('src/tweet-json.txt', mode='w') as output:   
    for tweet_id in query_ids:
        curr += 1
        # Progress message
        print("Retrieving: " + str(tweet_id) + 
              "  Missing: " + str(tweets- curr))
        try:
            # Get the tweet query status
            # Pass in 'extended' to get non truncated tweet text
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet.json, output)
            output.write("\n")
        # In case that Twitter responds with an error
        except tweepy.TweepError as e:
            print("Failure")
            # We make a dictionary with the tweets that throwed errors
            # for future handling
            failed_tweets[tweet_id] = e
            pass
end = timer()
print("Total Time: " + str(end-start))
print("Errors:")
print(failed_tweets)

In [16]:
# Now we read the dumped txt file, and append each tweet in a list
tweets = []
for line in open('src/tweet-json.txt', 'r'):
    tweets.append(json.loads(line))
# We verify the amount of tweets
len(tweets)

2354

In [19]:
# Then we explore the structure of the tweet in order to find the information that we want:
tweets[0]

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'id': 892420643555336193,
 'id_str': '892420643555336193',
 'full_text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 'truncated': False,
 'display_text_range': [0, 85],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 892420639486877696,
    'id_str': '892420639486877696',
    'indices': [86, 109],
    'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'url': 'https://t.co/MgUWQ76dJU',
    'display_url': 'pic.twitter.com/MgUWQ76dJU',
    'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
    'type': 'photo',
    'sizes': {'large': {'w': 540, 'h': 528, 'resize': 'fit'},
     'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'small': {'w': 540, 'h': 528, 'resize': 'fit'},
     'medium': {'w': 

> Once the tweets have been appended to the list, we extract the additional data that we want so we can build a dictionary, and finally construct the dataframe. The variables that will be extracted are:
- *id*
- *retweet_count*
- *favorite_count*

In [20]:
# We create lists that will hold each variable of the tweets
tweet_ids = []
retweet_counts = []
favorite_counts = []

for tweet in tweets:
    tweet_ids.append(tweet['id'])
    retweet_counts.append(tweet['retweet_count'])
    favorite_counts.append(tweet['favorite_count'])
    
# Now we construct the dataframe from a dictionary with the previous data
data = {'tweet_id':tweet_ids, 'retweet_count':retweet_counts, 'favorite_count':favorite_counts} 
tweets_counts = pd.DataFrame(data)
tweets_counts.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


> Now we have all the sources and data required for this project, we proceed to assess each dataset in order to find quality and tideness issues.

<a id='assessing'></a>
## Assesing Data

### Visual Assessment
The very first step when assessing the data is to do it visually, even though this might not be effective for large datasets, it will help us to get acquainted with the data.

#### Twitter Archive Dataset

In [21]:
# Display the twitter archive dataframe
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


> Indentified issues:
- Missing values for columns: (retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, in_reply_to_status_id, in_reply_to_user_id).
- There is no information about the type of dog, 'None' value for columns: (doggo, floofer, pupper, puppo).
- `source` column has embedded HTML code.

`twitter_archive` columns:
- **tweet_id**: is the tweet identifier.
- **in_reply_to_status_id**: reply status identifier.
- **in_reply_to_user_id**: reply to user identifier.
- **timestamp** the timestamp assigned to each tweet when it is created.
- **source**: the source (mobile) from which the tweet was made.
- **text**: the text (body) that each tweet has (during that time Twitter only allowed texts of 140 characters max).
- **retweeted_status_id**: identifier of the status if this tweet was retweeted.
- **retweeted_status_user_id**: identifier of the user if this tweet was retweeted.
- **retweeted_status_timestamp**: the timestamp given if this tweet was retweeted.
- **expanded_urls**: expanded urls.
- **rating_numerator**: numerator of the ranking, almost always above 10.
- **rating_denominator**: denominator of the ranking, fixed to 10.
- **name**: the given name to the dog.
- **doggo**: a type of dog. A big pupper, usually older. This label does not stop a doggo from behaving like a pupper.
- **floofer**: a type of dog. A any dog really. However, this label is commonly given to dogs with semmingly excess fur.
- **pupper**: a type of dog. A small doggo, usually younger. Can be equally, if not more mature than some doggos.
- **puppo**: a type of dog. A transitional state between pupper and doggo. Easily understood as the dog equivalent of a teenager.

**Definitions for dogs were taken from:** *TheDogtionary (via the #WeRateDogs book on Amazon)*

In [22]:
# Display the image predictions dataframe
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


> Indentified issues:
- Some breeds are capitalized while others not, also instead of a space the character '_' is used, for columns: (p1, p2, p3)